Vamos a cargar el dataset de AirBnB

In [1]:
airbnb<-read.csv('airbnb-listings.csv',sep = ';')
options(repr.plot.height=4,repr.plot.width=6,repr.plot.res = 300)

Warning message in file(file, "rt"):
“cannot open file 'airbnb-listings.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


Vamos a quedarnos con las columnas de mayor interés y convertimos de Square.Feet a Square.Meters (1 foot=0.092903m²):

In [ ]:
library(tidyverse)

df_airbnb <- airbnb %>% 
select(c('City','Room.Type','Neighbourhood','Accommodates','Bathrooms','Bedrooms','Beds',
               'Price','Square.Feet',
                'Guests.Included','Extra.People',
               'Review.Scores.Rating','Latitude', 'Longitude')) %>% 
mutate(Square.Meters=Square.Feet*0.092903) %>%
select(-c("Square.Feet"))

#Lo mismo pero con R básico:
#df_airbnb<-airbnb[,c('City','Room.Type','Neighbourhood','Accommodates','Bathrooms','Bedrooms','Beds',
#               'Price','Square.Feet',
#                'Guests.Included','Extra.People',
#               'Review.Scores.Rating','Latitude', 'Longitude')]
#df_airbnb$Square.Meters<-df_airbnb$Square.Feet*0.092903

Nos quedarmos solo con las entradas de Madrid para Room.Type=="Entire home/apt" y cuyo barrio (Neighbourhood) no está vacio ''
Podemos eliminar las siguientes columnas que ya no son necesarias:
"Room.Type",'City'

Llama a nuevo dataframe df_madrid.

In [ ]:

df_madrid<-df_airbnb %>% 
    filter(Room.Type=="Entire home/apt" & City=='Madrid' & Neighbourhood!='') %>% 
    select(-c("Room.Type",'City')) %>%
    droplevels()

set.seed(123)

In [ ]:
summary(df_madrid)

¿Que porcentaje de los apartamentos no muestran los metros cuadrados? Es decir que tienen NA en Square.Meters

In [ ]:
paste0("El porcentaje de NAs es: ",sum(is.na(df_madrid$Square.Meters)/nrow(df_madrid))*100,"%")

De todos los apartamentos que tienen un valor de metros cuadrados diferente de NA 
¿Que porcentaje de los apartamentos tienen 0 metros cuadrados?

In [ ]:
paste0("El porcentaje de 0s es: ",sum(df_madrid$Square.Meters==0,na.rm=T)/sum(!is.na(df_madrid$Square.Meters))*100,"%")

Reemplazar todos los 0m^2 por NA 

In [ ]:
df_madrid<-df_madrid %>% mutate(Square.Meters = ifelse(Square.Meters==0, NA, Square.Meters))

Son muchos, vamos a intentar crear un modelo que nos prediga cuantos son los metros cuadrados en función del resto de variables para tratar de rellenar esos NA.

Antes de eso deberíamos pintar el histograma de los metros cuadrados y ver si tenemos que filtrar algún elemento más.

In [ ]:
df_madrid %>% ggplot(aes(x=Square.Meters))+geom_histogram(bins=100)

In [ ]:
df_madrid %>% filter(Square.Meters<10)

# Existen muchos apartamentos con 0 m^2 y 
# hay algunos que tienen varias habitaciones y dicen que tienen menos de 20m^2


Asigna el valor NA a la columna Square.Meters de los apartamentos que tengan menos de 20 m^2

In [ ]:
df_madrid<-df_madrid %>% mutate(Square.Meters = ifelse(Square.Meters<20, NA, Square.Meters))
# Esa línea es equivalente a:
#df_madrid[,df_madrid$Square.Meters<20]<-NA

Existen varios Barrios que todas sus entradas son NA, vamos a eliminar del dataset todos los pisos que pertenecen a estos barrios.

In [ ]:
neighb_all_na <- df_madrid %>% group_by(Neighbourhood) %>% 
  summarise(num_na=sum(is.na(Square.Meters))/length(Square.Meters)) %>% 
  filter(num_na==1) %>% select(Neighbourhood)

In [ ]:
sum(apply(df_madrid,1,function(x) any(is.na(x))))

In [ ]:
dim(df_madrid[,c("Neighbourhood","Square.Meters")])
unique(na.omit(df_madrid[,c("Neighbourhood","Square.Meters")])$Neighbourhood)
#dim(na.omit(df_madrid))

In [ ]:
#Eliminamos los barrios que tienen todo NA

neighb_all_na <- df_madrid %>% group_by(Neighbourhood) %>% 
  summarise(num_na=sum(is.na(Square.Meters))/length(Square.Meters)) %>% 
  filter(num_na==1) %>% select(Neighbourhood)
df_madrid <- df_madrid[!df_madrid$Neighbourhood %in% neighb_all_na$Neighbourhood,]
paste("Quedan ",length(unique(df_madrid$Neighbourhood)),"barrios")
dim(df_madrid)

El barrio parece ser un indicador importante para los metros cuadrados de un apartamento.

Vamos a agrupar los barrios por metros cuadrados. Podemos usar una matriz de similaridad de Tukey tal y como hicimos en el curso de estadística:

In [ ]:
tky<-TukeyHSD(aov( formula=Square.Meters~Neighbourhood, data=df_madrid ))
tky.result<-data.frame(tky$Neighbourhood)
cn <-sort(unique(df_madrid$Neighbourhood))
resm <- matrix(NA, length(cn),length(cn))
rownames(resm) <- cn
colnames(resm) <- cn
resm[lower.tri(resm) ] <- round(tky.result$p.adj,4)
resm[upper.tri(resm) ] <- t(resm)[upper.tri(resm)] 
diag(resm) <- 1
library(ggplot2)
library(reshape2)
dfResm <- melt(resm)
ggplot(dfResm, aes(x=Var1, y=Var2, fill=value))+
  geom_tile(colour = "black")+
  scale_fill_gradient(low = "white",high = "steelblue")+
  ylab("Class")+xlab("Class")+theme_bw()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1),legend.position="none")

Usando como variable de distancia: 1-resm
Dibuja un dendrograma de los diferentes barrios.


In [ ]:
f_dist<-as.dist(1-resm)
hc <- hclust(f_dist,method="complete")
hcd <- as.dendrogram(hc)

In [ ]:
plot(hcd)

In [ ]:
library(dendextend)
#labels(hcd)<-iris$Species[labels(hcd)]
#hcd<-set(hcd,"labels_cex", 0.45) 
hcd <- as.dendrogram(hc)
hcd<-set(hcd,"labels_cex", 0.45) 
plot(color_branches(hcd,h=0.9),horiz=TRUE)


¿Que punto de corte sería el aconsejable?, ¿cuantos clusters aparecen?

In [ ]:
#Como hemos eliminado barrios parece que el mejor corte es a 0.2
ct<-cutree(hc,h=0.2)

In [ ]:
print("Vemos como en el modelo los clusters en general son buenos.")
library(cluster)
sil<-silhouette(ct,f_dist)
plot(sil,border=NA)

Vamos a crear una nueva columna en el dataframe df_madrid con un nuevo identificador marcado por los clusters obtenidos. Esta columna la llamaremos neighb_id

In [ ]:
df_barrios<-data.frame(names=names(ct),neighb_id=paste0("Neighb_id_",ct))
head(df_barrios)

In [ ]:
df_madrid_id <- df_madrid %>% 
    inner_join(df_barrios,by=c("Neighbourhood"='names')) %>%
    filter(!is.na(Square.Meters))
# El inner join es equivalente a:
#df_madrid<-merge(df_madrid,df_barrios,by.x = "Neighbourhood",by.y='names')

# Y esto sería equivalente a:
#df_madrid$neighb_id <- ct[df_madrid$Neighbourhood]

dim(df_madrid_id) 
head(df_madrid_id)

Vamos a crear dos grupos, uno test y otro train.

Tratamos de predecir los metros cuadrados.

In [ ]:
set.seed(12)
idx<-sample(1:nrow(df_madrid_id),nrow(df_madrid_id)*0.8)
df_madrid_id_train<-df_madrid_id[idx,]
df_madrid_id_test<-df_madrid_id[-idx,]

In [ ]:
model<-lm(formula=Square.Meters~neighb_id+neighb_id+Price+Bedrooms, df_madrid_id_train)
#model<-lm(formula=Square.Meters~neighb_id+Bedrooms+Price:Accommodates, df_madrid_id_train) # Este modelo hace overfitting
#model<-lm(formula=Square.Meters~neighb_id+Bedrooms+Price, df_madrid_id_train)
summary(model)

In [ ]:
print("Comprobamos en el R² y otras medidas en train y test para ver si tenemos overfitting:")
caret::postResample(predict(model,df_madrid_id_train),obs = df_madrid_id_train$Square.Meters)
caret::postResample(predict(model,df_madrid_id_test),obs = df_madrid_id_test$Square.Meters)

In [ ]:
plot(model$model$Square.Meters,model$residual)
# Parece que tenemos un poco de heterocedasticidad, la varianza parece aumentar ligeramente, pero no es preocupante
# También vemos un outlier que podemos tratar.

In [ ]:
hist(model$residual,breaks=20)

In [ ]:
print("Tenemos un elemento que tiene un residuo muy alto, ¿nos afectará a la predicción?")
plot(cooks.distance(model))

In [ ]:
cook_d<-cooks.distance(model)

df_madrid_id_train[names(cook_d),] %>% filter(cook_d>0.2)
# La línea anterior es equivalente a:
#df_madrid_id_train[names(which(cook_d>0.2)),]

In [ ]:
head(cook_d)

In [ ]:
# Aquí usamos como índice names(cook_d) porque los NAs no aparecen en la variable cook_d, por eso los indices no tienen porque coincidir
model_cook<-lm(formula=Square.Meters~neighb_id+Price+Bedrooms, df_madrid_id_train[names(cook_d),] %>% filter(cook_d<0.2))
print("Comprobamos en el R² y otras medidas en train y test para ver si tenemos overfitting:")
#caret::postResample(predict(model_cook,df_madrid_id_train),obs = df_madrid_id_train$Square.Meters)
#caret::postResample(predict(model_cook,df_madrid_id_test),obs = df_madrid_id_test$Square.Meters)

No podemos filtrar por la distancia de cooks porque eliminamos un grupo entero de barrios y daría error.

In [ ]:
plot(cooks.distance(model_cook))

In [ ]:
plot(model_cook$model$Square.Meters,model_cook$residual)

Vamos a probar con regularización por Ridge

In [ ]:
library(glmnet)
over_fit_model<-lm(formula=Square.Meters~neighb_id+Bedrooms+Price*Accommodates, df_madrid_id)
X<-model.matrix(over_fit_model)
y<-as.matrix(over_fit_model$model$Square.Meters,ncols=1)
set.seed(12)
idx<-sample(1:nrow(X),nrow(X)*0.8)
X_train<-X[idx,]
X_test<-X[-idx,]
y_train<-y[idx,]
y_test<-y[-idx,]

cvfit<-cv.glmnet(X_train,y_train,nfolds=10,alpha=0)
cvfit$lambda.1se
cvfit$lambda.min
plot(cvfit)

In [ ]:
gmodel<-glmnet(X,y,alpha=0,lambda = cvfit$lambda.1se)
caret::postResample(predict(gmodel,X_train),obs = y_train)
caret::postResample(predict(gmodel,X_test),obs = y_test)
# Aunque nos sale un R^2 muy bueno si miramos los residuos nos daremos cuenta que nuestro modelo es bastante malo
# a Medida que aumenta el valor de los metros cuadrados el error crece de forma lineal.
plot(y_train,y_train-predict(gmodel,X_train))

In [ ]:
# Vamos a usar un valor más bajo de lambda:
gmodel<-glmnet(X,y,alpha=0,lambda = cvfit$lambda.min)
caret::postResample(predict(gmodel,X_train),obs = y_train)
caret::postResample(predict(gmodel,X_test),obs = y_test)
plot(y_train,y_train-predict(gmodel,X_train))
hist(y_train-predict(gmodel,X_train),breaks=20)

Después de todas las pruebas concluimos que el mejor modelo era el primero, el más sencillo como veremos más adelante comprobando sus residuos. 

Aunque con tan pocas muestras es dificil saber cual es el mejor modelo. Una regla buena es que en igualdad de condiciones, el modelo más sencillo suele ser el más fiable (navaja de Ockham). 

Mirad el histograma de los residuos sobre el conjunto de test para evaluar la calidad de vuestro modelo

In [ ]:
df_madrid_id_test$pred<-model %>% predict(df_madrid_id_test)
hist(df_madrid_id_test$Square.Meters-df_madrid_id_test$pred,breaks=15)


In [ ]:
# Los residuos no siguen una distribución muy gaussiana, aunque con tan pocas muestras es dificil de verificarlo.
# Al menos su diagrama de dispersión si tiene buena pinta, no se ven tendencias:
plot(df_madrid_id_test$pred,df_madrid_id_test$Square.Meters-df_madrid_id_test$pred)

In [ ]:
library(caret)
postResample(df_madrid_id_test$pred,obs = df_madrid_id_test$Square.Meters)

In [ ]:
hist(df_madrid_id_test$pred)
hist(df_madrid_id_test$Square.Meters)

Si tuvieramos un anuncio de un apartamento para 6 personas (Accommodates), con 1 baño, con un precio de 80€/noche y 3 habitaciones en el barrio de Sol, con 3 camas y un review de 80.
¿Cuantos metros cuadrados tendría? Si tu modelo necesita algúna variable adicional puedes inventartela dentro del rango de valores del dataset.
¿Como varía sus metros cuadrados con cada habitación adicional?

In [ ]:
# Primero buscamos el neighb_id del barrio de Sol:
neighb_id_sol<-df_barrios %>% filter(names=="Sol") %>% select(neighb_id)
paste0("El id de barrio es:",neighb_id_sol)
# Luego creamos el dataframe:
df_appartment<-data.frame(neighb_id=neighb_id_sol,Bedrooms=3,Price=80,Accommodates=6)
# Por último damos la predicción
pred_m2<-predict(model,df_appartment)
paste("Los metros cuadrados son:",round(pred_m2))

In [ ]:
cf<-coefficients(model)
cf_bedroom_sol<-cf['Bedrooms']

paste("En media cada habitación aumenta el tamaño del apartemos en:",round(cf_bedroom_sol,2),"m^2")


Rellenar los Square.Meters con valor NA con el estimado con el modelo anterior.

In [ ]:
df_madrid_id_all <- df_madrid %>% inner_join(df_barrios,by=c("Neighbourhood"='names')) 

df_madrid_id_all$Square.Meters[is.na(df_madrid_id_all$Square.Meters)]<-
 round(predict(model,df_madrid_id_all[is.na(df_madrid_id_all$Square.Meters),]))

head(df_madrid_id_all)

Usar PCA para encontrar el apartamento más cercano a uno dado.

Este algoritmo nos ayudaría a dado un apartamento que el algoritmo nos devolvería los 5 apartamentos más similares.


Crearemos una función tal que le pasemos un apartamento con los siguientes datos:
* Accommodates	
* Bathrooms	
* Bedrooms	
* Beds	
* Price	
* Guests.Included	
* Extra.People	
* Review.Scores.Rating	
* Latitude	
* Longitude	
* Square.Meters

y nos devuelva los 5 más similares de:

In [ ]:
df_madrid_pca<-na.omit(df_madrid_id_all[,c("Accommodates","Bathrooms","Bedrooms","Latitude","Longitude",
                                           "Beds","Price","Review.Scores.Rating","Square.Meters",'neighb_id')])

# Aunque podríamos dividir los datos entre test y training, aquí realmente no estamos tratando de hacer una 
# predicción, sino tratando de encontrar en el mismo dataset los elementos más similares.
pca_df<-prcomp(df_madrid_pca %>% select(-neighb_id),center = TRUE, scale. = TRUE)

In [ ]:
summary(df_madrid_pca)

In [ ]:
plot(pca_df$sdev^2/sum(pca_df$sdev^2),main="Autovalores")

In [ ]:
str(pca_df)

In [ ]:
get_closest_element<-function(pca_df, new_flat,num_flats){
    pca_new<-predict(pca_df,newdata = new_vector)
    pca_orig<-pca_df$x[,1:2]
    pca_new <-pca_new[,1:2]

    idx<-order(rowSums((pca_new-pca_orig)^2))
    df_madrid_pca[idx %in% 1:num_flats,]
}

In [ ]:
new_vector<-(df_madrid_pca %>% select(-neighb_id))[10,]
new_vector
get_closest_element(pca_df,new_vector,5)

Bonus: ¿de todas las coordenadas PCA cual es la que mejor permite clasificar por id de barrio?

¿Por qué?

##### Forma 1

In [ ]:
# Realizadmos un test de ANOVA de cada componente principal (PC) por neighb_id

pca_x_neighb<-cbind(data.frame(pca_df$x),neighb_id=df_madrid_pca$neighb_id)
pca_cols<-colnames(pca_x_neighb)[1:ncol(pca_x_neighb)-1]
df_F_val<-data.frame(pca=pca_cols,F=NA)
for (idx in (1:nrow(df_F_val))){
    formula<-paste(pca_cols[idx],"~ neighb_id")
    df_F_val$F[idx]<-oneway.test(pca_x_neighb,formula = as.formula(formula))$statistic
}
df_F_val

Parece que PC6 y PC9 son los que más varían a la hora de elegir por id de barrio. Vamos a analizar porqué.

In [ ]:
library(ggpubr)
g1<-ggplot(data.frame(pca_df$x),aes(x=PC6,color=df_madrid_pca$neighb_id))+geom_density(size=1)
g2<-ggplot(data.frame(pca_df$x),aes(x=PC9,color=df_madrid_pca$neighb_id))+geom_density(size=1)

ggarrange(g1,g2,ncol=1)

In [ ]:
pca_df$rotation
# Vemos como la contribución principal a PC6 y PC9 son los metros cuadrados 
# o medidas relacionadas con los mismos como el número de habitaciones.
# Lo cual es lógico porque hemos clusterizado utilizando los metros cuadrados.

##### Forma 2

Otra forma de obtener lo mismo con LDA

In [ ]:
MASS::lda(formula=neighb_id~.,data=pca_x_neighb)

Si nos fijamos en los coeficiente, en "Proportion of trace" vemos que con los dos primeros componentes LD1 y LD2 son los más importantes para clasificar por neighb_id.
Y si miramos los coeficientes que más importancia tienen en LD1 y LD2 son a su vez PC2 y PC9 como habíamos visto antes con el test ANOVA.